In [1]:
import math
from math import isnan, nan
from os import O_SYNC, kill
from tracemalloc import stop
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.integrate import quad
import scipy.interpolate as inter
import sys
import numba
from IPython.core.debugger import Pdb

In [2]:
# Calculate cross section considering Earth layers
@numba.jit(nopython=True, nogil=True)
def chi_calc(m_mu, E_mu, m_z, Z, A):
    me=0.511/1000 #GeV
    a=111*Z**(-1/3)/me
    d=0.164*A**(-2/3)
    t_a=(1/a)**2
    t_d=d
    #t_a = (2e-5)**2 #GeV^2
    #t_d = (67e-3)**2 #GeV^2
    t_min = m_z**4/(4*E_mu**2)
    t_max = m_z**2 + m_mu**2
    chi_max = t_d**2/(t_a - t_d)**3*((t_a - t_d)*(t_a + t_min)/(t_max + t_a) + (t_a - t_d)*(t_d + t_min)/(t_max + t_d) + (t_a + t_d + 2*t_min)*np.log((t_max+t_d)/(t_max+t_a)))
    chi_min = t_d**2/(t_a - t_d)**3*((t_a - t_d) + (t_a - t_d) + (t_a + t_d + 2*t_min)*np.log((t_min+t_d)/(t_min+t_a)))
    chi_IWW = Z**2*(chi_max - chi_min)
    return(chi_IWW)


In [3]:
@numba.jit(nopython=True, nogil=True)
def diff_cross_section(eps, alpha, m_z, E_mu, m_mu, Z, x, A):
    '''
        Calculate the differential cross section for DP production by muon Bremsstrahlung
        using the IWW approximation
    '''
    m_z = m_z/1000
    chi_IWW = chi_calc(m_mu, E_mu, m_z, Z, A)
    u_max = -m_z**2*(1-x)/x - m_mu**2*x
    u_min = -x*(E_mu*0.1)**2 + u_max #theta_max = 0.1 (E_mu(theta) = m_mu/theta)
    #print(u_max, u_min)
    dsig_dx_min = 2*eps**2*alpha**3*chi_IWW*np.sqrt((x**2 - (m_z/E_mu)**2))*(m_mu**2*x*(-2+2*x+x**2)-2*(3-3*x+x**2)*u_min)/(3*x*u_min**2)
    dsig_dx_max = 2*eps**2*alpha**3*chi_IWW*np.sqrt((x**2 - (m_z/E_mu)**2))*(m_mu**2*x*(-2+2*x+x**2)-2*(3-3*x+x**2)*u_max)/(3*x*u_max**2)
    dsig_dx = dsig_dx_max - dsig_dx_min

    return(dsig_dx)

In [4]:
#@numba.jit(nopython=True, nogil=True)
def ExtractData(text):
    '''
        This function separtes the data of an archive into diferent arrays
    '''    
    for i in range(len(text)):
        text[i] = text[i].split("\t")
        text[i] = [float(x) for x in text[i]]
    
    return(text)


In [5]:
#@numba.jit(nopython=True, nogil=True)
def Build_zenital(text, zenit, ratio, N):
    '''
        Build the arrays of zenital angles and the muon ratios in this direction 
    '''
    for i in range(N):
        zenit[i]=np.arccos(text[i][0])
        ratio[i]=text[i][1]

In [6]:
#@numba.jit(nopython=True, nogil=True)
def Build_general(text, list1, list2, N):

    for i in range(N):
        list1[i]=text[i][0]
        list2[i]=text[i][1]

In [7]:
def ExtractData1(text):
    '''
        This function separtes the data of an archive into diferent arrays
    '''    
    for i in range(len(text)):
        text[i] = text[i].split("  ")
        text[i] = [float(x) for x in text[i]]
    
    return(text)

In [8]:
def InterpMassAtt(element):

    # Import sodium mass attenuation
    #arq = open('/home/dfreitas/DPnum_Comp/Oscillation/Mass_att(Exmu)_%s.txt' %(element), 'r') 
    arq = open('/home/davidc/Documents/Master\'s Analisys/Parameter space/Codes/Oscillation/Mass_att(Exmu)_%s.txt' %(element), 'r') 
    texto = arq.readlines()
    arq.close()

    texto = ExtractData1(texto)

    N1=len(texto)
    en=np.zeros(N1)
    mu=np.zeros(N1)

    for i in range(N1):
        en[i]=texto[i][0]
        mu[i]=texto[i][2]
    func=inter.interp1d(en,mu)

    return(func, en)

In [9]:
def GatherEnergies(en_A,en_B):

    en=[]
    enfinal=[]
    for i in range(len(en_A)):
        en.append(en_A[i])
    for i in range(len(en_B)):
        en.append(en_B[i])
    for i in range(len(en)):
        for j in range(i+1,len(en)):
            if (en[i]==en[j]):
                en[i]=0.0
    for i in range(len(en)):
        if (en[i]!=0.0):
            enfinal.append(en[i])  
    
    return(enfinal)

In [10]:
def MuonCalc(Ei, theta, E, mDP, x, kappas, Nmuabove100, Na):
    '''
    Function to compute the number NDPs_tot of muon Bremsstralung at the Earth
    producing DPs at each step j and angle t, for each coupling parameter and a
    DP mass mDP.
    '''
    alpha1=1/137
    m_mu=105.7/1000 #muon mass (GeV)
    rho_mantle = 4 #mantle density (g/cm³) -> DOI: 10.1103/PhysRevLett.128.171801
    L_muon = np.diff(x)[0]/rho_mantle #step length inside the mantle for muon bremsstrahlung (cm)
    A_m = 24.5 #Mass number of the mantle (GeV) -> DOI: 10.1103/PhysRevLett.128.171801
    
    N=50
    Z_mantle = 10.76
    dsig_dx_m=np.zeros((len(Ei),N))
    sig_m=np.zeros((len(Ei),len(x)))
    ratio=np.zeros((len(Ei),N))

    #FIRST WE COMPUTE THE PROCESSES IN THE EARTH
    NDPs_tot = np.zeros((len(theta), N)) #Number of DPs generated in each angle and energy
    
    #DP energies (GeV)
    if (mDP*1e-3>1E-06):
        E_z = np.linspace(mDP*1e-3, 0.1, N) #GeV
                    
    else:
        E_z = np.linspace(1E-06, 0.1, N) #GeV

    #FIRST, WE COMPUTE THE NUMBER OF DPs GENERATED BY MUON BREMSSTRAHLUNG IN EACH EARTH LAYER
    for i in range(len(Ei)-1):
        print("Bremsstrahlung = %f\n" %(np.sum(NDPs_tot)), i)
        if (i>=900): continue
        for t in range(0,len(theta)):           
            for j in range(0,len(x)-1): 

                if ((14000-x[j]/rho_mantle/100)>=0.0):
                    if (E[i][j]<1*10**2): continue
                    for r in range(N):
                        ratio[i][r] = E_z[r]/(E[i][j]/1000)
            
                        if (r!=0):

                            #Bremsstrahlung
                            dsig_dx_m[i][r] = diff_cross_section(kappas, alpha1, mDP, E[i][j]/1000, m_mu, Z_mantle, (ratio[i][r]+ratio[i][r-1])/2, A_m)
                            integr=(dsig_dx_m[i][r]+dsig_dx_m[i][r-1])*(ratio[i][r]-ratio[i][r-1])/2
                            sig_m[i][j]=integr*(1.973*10**(-16))**2*10**4*10**24*10**12 #pb
                            xsec=sig_m[i][j]*10**(-12)*10**(-24) #cm^2
                            Nbremss=Nmuabove100[i][t][j]*rho_mantle*Na/A_m*L_muon*xsec*60*60*24*365.25 # num of DP's per year
                            NDPs_tot[t][r] += Nbremss
                            
    return(NDPs_tot)

In [15]:
def CalcEvents(NDPs_tot, theta, mDP, kappas, Na, epsilon, funcmantle, funcore, R_earth):

    alpha1=1/137
    me=0.511/1000 #electron mass (GeV)
    rho_mantle = 4 #mantle density (g/cm³) -> DOI: 10.1103/PhysRevLett.128.171801
    rho_core = 11  #core density (g/cm³) -> DOI: 10.1103/PhysRevLett.128.171801
    rho_NaI = 3.67 #NaI density (g/cm³)
    R_core = 3471000 #core's radius in (m)
    A_m = 24.5 #Mass number of the mantle (GeV) -> DOI: 10.1103/PhysRevLett.128.171801
    A_c = 54   #Mass number of the core (GeV) -> DOI: 10.1103/PhysRevLett.128.171801

    N=50
    Z_mantle = 10.76
    Z_core = 26.2
    
    #FIRST WE COMPUTE THE PROCESSES IN THE EARTH
    Ndec_res = np.zeros((len(theta), 9000, N)) #Number of decays in each angle, step and energy in Earth
    Ndec_tot = np.zeros((len(theta), 9000, N)) #Number of decays in each angle, step and energy in crystals
    q = 0.30282212088 #Elementary charge in natural units
    alpha = q**2/(4*np.pi) #Fine structure constant
    alphaline = (q*kappas)**2/(4*np.pi)

    ne_m=(rho_mantle/(A_m/Na))*(1.973*10**(-14))**3   #Electron number density for mantle (in GeV^3)
    ne_c=(rho_core/(A_c/Na))*(1.973*10**(-14))**3     #Electron number density for core (in GeV^3)
    ne_NaI=rho_NaI/(149.89/Na)*(1.973*10**(-14))**3   #Electron number density for NaI crystals (in GeV^3)
    mgamma_m=(4*np.pi*alpha1*ne_m/me)**0.5            #effective photon mass for mantle
    mgamma_c=(4*np.pi*alpha1*ne_c/me)**0.5            #effective photon mass for core

    #Compton-like scaterring Cross-Section
    # with open('/home/dfreitas/DPnum_Comp/Comp-xs2/CSxE_m=%f_k=%s00.txt' %(mDP, epsilon), 'r') as archive1:
    #         arqComp = archive1.read()

    with open('/home/davidc/Documents/Master\'s Analisys/Results Compton/Cross Section 2/CSxE_m=%f_k=%s00.txt' %(mDP, epsilon), 'r') as archive1:
            arqComp = archive1.read() 
        
    linesComp = arqComp.split("\n")
    linesComp = linesComp[:-1]

    for row in range(0, len(linesComp)):
        linesComp[row] = linesComp[row].split("   ")
        linesComp[row] = [float(x) for x in linesComp[row]]
    
    sigComparq_m = np.zeros(len(linesComp))
    sigComparq_c = np.zeros(len(linesComp))
    sigComparqNaI = np.zeros(len(linesComp))
    E_kComp = np.zeros(len(linesComp))

    for l in range(0, len(linesComp)):
        E_kComp[l] = linesComp[l][0]
        sigComparq_m[l] = (Z_mantle/64)*linesComp[l][1]*(1e-24)*(5.06e13)**2 #GeV⁻²/atom
        sigComparq_c[l] = (Z_core/64)*linesComp[l][1]*(1e-24)*(5.06e13)**2 #GeV⁻²/atom
        sigComparqNaI[l] = linesComp[l][1]*(1e-24)*(5.06e13)**2 #GeV⁻²/atom
    
    CScompfunc_m = inter.interp1d(E_kComp, sigComparq_m, kind = 'cubic', fill_value="extrapolate")
    CScompfunc_c = inter.interp1d(E_kComp, sigComparq_c, kind = 'cubic', fill_value="extrapolate")

    #Absorption Cross-Section
    with open('/home/davidc/Documents/Master\'s Analisys/Parameter space/Codes/Absorption/PeXSMantle.txt', 'r') as archivem:
            arqAbsm = archivem.read()
    with open('/home/davidc/Documents/Master\'s Analisys/Parameter space/Codes/Absorption/PeXSCore.txt', 'r') as archivec:
            arqAbsc = archivec.read() 
    # with open('/home/dfreitas/DPnum_Comp/Absorption/PeXSMantle.txt', 'r') as archivem:
    #         arqAbsm = archivem.read()
    # with open('/home/dfreitas/DPnum_Comp/Absorption/PeXSCore.txt', 'r') as archivec:
    #         arqAbsc = archivec.read()  

    linesAbsm = arqAbsm.split("\n")
    linesAbsc = arqAbsc.split("\n")
    linesAbsm = linesAbsm[:-1]
    linesAbsc = linesAbsc[:-1]
    
    for row in range(0, len(linesAbsm)):
        linesAbsm[row] = linesAbsm[row].split(" ")
        linesAbsm[row] = [float(x) for x in linesAbsm[row]]
    
    for row in range(0, len(linesAbsc)):
        linesAbsc[row] = linesAbsc[row].split(" ")
        linesAbsc[row] = [float(x) for x in linesAbsc[row]]

    sigAbs_m = np.zeros(len(linesAbsm))
    sigAbs_c = np.zeros(len(linesAbsc))
    E_kAbsm = np.zeros(len(linesAbsm))
    E_kAbsc = np.zeros(len(linesAbsc))

    for l in range(0, len(linesAbsm)):
        E_kAbsm[l] = linesAbsm[l][0]
        if mDP < E_kAbsm[l]:
            sigAbs_m[l] = (linesAbsm[l][1]*(5.06e13)**2/np.sqrt(1-mDP**2/E_kAbsm[l]**2))*(alphaline/alpha)*(A_m/Na) #GeV⁻²/atom (1/(Na/A)))
            
    for l in range(0, len(linesAbsc)):
            E_kAbsc[l] = linesAbsc[l][0]
            if mDP < E_kAbsc[l]:
                sigAbs_c[l] = (linesAbsc[l][1]*(5.06e13)**2/np.sqrt(1-mDP**2/E_kAbsc[l]**2))*(alphaline/alpha)*(A_c/Na) #GeV⁻²/atom (1/(Na/A)))

    CSabsfuncm = inter.interp1d(E_kAbsm, sigAbs_m, kind = 'linear', fill_value="extrapolate")
    CSabsfuncc = inter.interp1d(E_kAbsc, sigAbs_c, kind = 'linear', fill_value="extrapolate")
    CScompfuncNaI = inter.interp1d(E_kComp, sigComparqNaI, kind = 'cubic', fill_value="extrapolate")

    with open('/home/davidc/Documents/Master\'s Analisys/Parameter space/Codes/Absorption/PeXSNaI.txt', 'r') as archive3:
            arqAbs1 = archive3.read() 
    # with open('/home/dfreitas/DPnum_Comp/Absorption/PeXSNaI.txt', 'r') as archive3:
    #         arqAbs1 = archive3.read()

    linesAbs1 = arqAbs1.split("\n")
    linesAbs1 = linesAbs1[:-1]
    
    for row in range(0, len(linesAbs1)):
        linesAbs1[row] = linesAbs1[row].split(" ")
        linesAbs1[row] = [float(x) for x in linesAbs1[row]]

    sigAbsNaI = np.zeros(len(linesAbs1))
    E_kAbs = np.zeros(len(linesAbs1))

    for l in range(0, len(linesAbs1)):
        E_kAbs[l] = linesAbs1[l][0]
        if mDP < E_kAbs[l]:
            sigAbsNaI[l] = (linesAbs1[l][1]*(5.06e13)**2/np.sqrt(1-mDP**2/E_kAbs[l]**2))*(alphaline/alpha)*(149.89/Na) #GeV⁻²/atom = (1/(Na/A))) barns/atom

    CSabsfuncNaI = inter.interp1d(E_kAbs, sigAbsNaI, kind = 'linear', fill_value="extrapolate")

    #DP energies (GeV)
    if (mDP*1e-3>1E-06):
        E_z = np.linspace(mDP*1e-3, 0.1, N) #GeV
                    
    else:
        E_z = np.linspace(1E-06, 0.1, N) #GeV

    #NOW WE COMPUTE THE NUMBER OF STINGUISHED DPs INSIDE THE EARTH (DOESN'T WORK COMPUTING TOGETHER WITH THE BREMSSTRAHLUNG)
    sigCompm = (kappas/epsilon)**2*CScompfunc_m(1e03*E_z) #Compton cross-section
    sigAbsm = CSabsfuncm(1e03*E_z)  #Absorption cross section
    sigCompc = (kappas/epsilon)**2*CScompfunc_c(1e03*E_z) #Compton cross-section
    sigAbsc = CSabsfuncc(1e03*E_z)  #Absorption cross section
    sigAbsNaI = CSabsfuncNaI(E_z*1000)  #Absorption cross section in NaI
    sigCompNaI = (kappas/epsilon)**2*CScompfuncNaI(E_z) #Compton cross-section
    
    for t in range(0,len(theta)): #angles
        d = R_earth*np.sqrt(0.5*(1+np.cos(theta[t]))) #distance from the center to the sphere string
        print("events", t)
        if d >= R_core:
            xe = np.linspace(0,R_earth*np.sqrt(2*(1-np.cos(theta[t]))),9000) #Earth's path length (m)
            Le = np.diff(xe)[0]*100 #step length inside Earth (cm)
            print("\nmantle only, %f extinctions and %f detections" %(np.sum(Ndec_res),np.sum(Ndec_tot)))
            for j in range(0,len(xe)):  #steps inside rock
                if ((R_earth*np.sqrt(2*(1-np.cos(theta[t])))-xe[j])>=0.0):
                    
                    for r in range(0, N):

                        if r!= 0:

                            Egamma=(E_z[r]+E_z[r-1])/2 #GeV
                            lambAbsm = 1/(ne_m*sigAbsm[r])         #absorption length in mantle
                            lambAbsNaI = 1/(ne_NaI*sigAbsNaI[r])   #absorption length in NaI
                            lambCompm = 1/(ne_m*sigCompm[r])       #Compton length in mantle
                            lambCompNaI = 1/(ne_NaI*sigCompNaI[r]) #Compton length in NaI

                            if mDP/1000 > 2*me: #Computing the number of decays + events near the detector
                                gamma=kappas**2*alpha1*(mDP*1e-3)/3*(1+2*me**2/(mDP*1e-3)**2)*(1-4*me**2/(mDP*1e-3)**2)**0.5
                                Ldec = np.sqrt(E_z[r]**2/(mDP*1e-3)**2 - 1)/gamma  #decay length (GeV^{-1})
                                dist = (R_earth*np.sqrt(2*(1-np.cos(theta[t])))-xe[j])*100*5.06e13 #distance from step j to th detector ((GeV^{-1}))
                                Pdetec = np.exp(-dist*(1/Ldec+1/lambAbsm+1/lambCompm))*(1-np.exp(-40*5.06e13*(1/Ldec+1/lambAbsNaI+1/lambCompNaI))) #detection probability (0.4 m detector) 
                                PEarth = 1 - np.exp(-Le*5.06e13*(1/lambAbsm+1/Ldec+1/lambCompm)) #Extinction probability in this step
                                if Pdetec<1e-13:
                                    Pdetec=0
                                Ndec_res[t][j][r] += PEarth*(NDPs_tot[t][r]) #extinction number in this step
                                Ndec_tot[t][j][r] += Pdetec*(NDPs_tot[t][r]-Ndec_res[t][j][r]) #detection number in this step via either direct
                                                                                        # interacion or decay  
                                if j>0: 
                                    if NDPs_tot[t][r] < PEarth*(round(NDPs_tot[t][r], 4)):
                                        print(PEarth*(round(NDPs_tot[t][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 dec", j)
                                        Ndec_res[t][j][r] -= np.abs(NDPs_tot[t][r] - PEarth*(round(NDPs_tot[t][r], 4)))
                                        print(PEarth*(round(NDPs_tot[t][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 dec", j)
                                        #Pdb().set_trace()
                                        
                                    if NDPs_tot[t][r] < Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j][r], 4)):
                                        print(Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 det", j)
                                        Ndec_tot[t][j][r] -= np.abs(NDPs_tot[t][r] - Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j][r], 4)))
                                        print(Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 det", j)
                                        #Pdb().set_trace()
                                else:
                                    if NDPs_tot[t][r] < Ndec_res[t][j][r]:
                                        print(Ndec_res[t][j][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 dec")
                                        Ndec_res[t][j][r] -= (np.abs(Ndec_res[t][j][r] - NDPs_tot[t][r]))
                                        print(Ndec_res[t][j][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 dec")
                                        #Pdb().set_trace()
                                    if NDPs_tot[t][r] < Ndec_tot[t][j][r]:
                                        print(Ndec_res[t][j][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 det")
                                        Ndec_tot[t][j][r] -= (np.abs(Ndec_tot[t][j][r] - NDPs_tot[t][r]))
                                        print(Ndec_res[t][j][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 det")
                                        #Pdb().set_trace()

                                NDPs_tot[t][r] -= (Ndec_tot[t][j][r]+Ndec_res[t][j][r])

                            else:
                                gamma = kappas**2*alpha1**4/(2**7*3**6*5**2*np.pi**3)*(mDP/1000)**9/me**8*(17/5+67/42*(mDP/1000)**2/me**2+128941/246960*(mDP/1000)**4/me**4)
                                Ldec = np.sqrt(E_z[r]**2/(mDP*1e-3)**2 - 1)/gamma
                                dist = (R_earth*np.sqrt(2*(1-np.cos(theta[t])))-xe[j])*100*5.06e13 #distance from step j to the detector ((GeV^{-1}))
                                Pdetec = np.exp(-dist*(1/Ldec+1/lambAbsm+1/lambCompm))*(1-np.exp(-40*5.06e13*(1/Ldec+1/lambAbsNaI+1/lambCompNaI))) #detection probability (0.4 m detector) 
                                if Pdetec<1e-13:
                                    Pdetec=0
                                PEarth = 1 - np.exp(-Le*5.06e13*(1/lambAbsm+1/Ldec+1/lambCompm)) #Extinction probability in this step
                                P=kappas**2*(mDP*1e-03)**4*funcmantle(Egamma*1000)*Le*5.06e13/(((mDP*1e-03)**2-mgamma_m**2)**2+Egamma**2*funcmantle(Egamma*1000)**2)
                                NOsc = P*(round(NDPs_tot[t][r], 4))#Oscillation
                                Ndec_res[t][j][r] += PEarth*(round(NDPs_tot[t][r]-NOsc, 4)) + NOsc #extinction number in this step
                                Ndec_tot[t][j][r] += Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j][r], 4)) #detection number in this step via either direct
                                                                                                         # interacion or decay 
                                if j>0: 
                                    if NDPs_tot[t][r] < PEarth*(round(NDPs_tot[t][r], 4)):
                                        print(PEarth*(round(NDPs_tot[t][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 dec", j)
                                        Ndec_res[t][j][r] -= np.abs(NDPs_tot[t][r] - PEarth*(round(NDPs_tot[t][r], 4)))
                                        print(PEarth*(round(NDPs_tot[t][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 dec", j)
                                        Pdb().set_trace()
                                        
                                    if NDPs_tot[t][r] < Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j][r], 4)):
                                        print(Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 det", j)
                                        Ndec_tot[t][j][r] -= np.abs(NDPs_tot[t][r] - Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j][r], 4)))
                                        print(Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 det", j)
                                        Pdb().set_trace()
                                else:
                                    if NDPs_tot[t][r] < Ndec_res[t][j][r]:
                                        print(Ndec_res[t][j][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 dec")
                                        Ndec_res[t][j][r] -= (np.abs(Ndec_res[t][j][r] - NDPs_tot[t][r]))
                                        print(Ndec_res[t][j][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 dec")
                                        Pdb().set_trace()
                                    if NDPs_tot[t][r] < Ndec_tot[t][j][r]:
                                        print(Ndec_res[t][j][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 det")
                                        Ndec_tot[t][j][r] -= (np.abs(Ndec_tot[t][j][r] - NDPs_tot[t][r]))
                                        print(Ndec_res[t][j][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 det")
                                        Pdb().set_trace()

                                NDPs_tot[t][r] -= (Ndec_tot[t][j][r]+Ndec_res[t][j][r]) #atualizing the number of total DPs
                                
        else:
            L_c = 2*np.sqrt(R_core**2-R_earth**2/2*(1+np.cos(theta[t]))) #length of the core path (m)
            L_m = R_earth*np.sqrt(2*(1-np.cos(theta[t])))/2 - L_c/2 #length of each mantle path (m)
            x_m1 = np.linspace(0,L_m,3000)
            x_c = np.linspace(L_m, (L_m+L_c),3000)
            x_m2 = np.linspace((L_c+L_m),(L_c+2*L_m),3000)
            L_core = np.diff(x_c)[0]*100
            L_m1 = np.diff(x_m1)[0]*100
            L_m2 = np.diff(x_m2)[0]*100
            
            print("\n mantle 1, %f extinctions and %f detections" %(np.sum(Ndec_res),np.sum(Ndec_tot)))
            for j in range(0,len(x_m1)): #steps inside rock

                #Before recahing the nucleus
                if (L_m - x_m1[j] >= 0):
                    for r in range(0, N):
                        if r!= 0:
                            Egamma=(E_z[r]+E_z[r-1])/2 #GeV
                            lambAbsm = 1/(ne_m*sigAbsm[r])         #absorption length in mantle
                            lambAbsNaI = 1/(ne_NaI*sigAbsNaI[r])   #absorption length in NaI
                            lambCompm = 1/(ne_m*sigCompm[r])       #Compton length in mantle
                            lambCompNaI = 1/(ne_NaI*sigCompNaI[r]) #Compton length in NaI
                                
                            if mDP/1000 > 2*me: #Computing the number of decays + events near the detector
                                gamma=kappas**2*alpha1*(mDP*1e-3)/3*(1+2*me**2/(mDP*1e-3)**2)*(1-4*me**2/(mDP*1e-3)**2)**0.5
                                Ldec = np.sqrt(E_z[r]**2/(mDP*1e-3)**2 - 1)/gamma
                                dist = (R_earth*np.sqrt(2*(1-np.cos(theta[t])))-x_m1[j])*100*5.06e13 #distance from step j to th detector ((GeV^{-1}))
                                Pdetec = np.exp(-dist*(1/Ldec+1/lambAbsm+1/lambCompm))*(1-np.exp(-40*5.06e13*(1/Ldec+1/lambAbsNaI+1/lambCompNaI))) #detection probability (0.4 m detector) 
                                if Pdetec<1e-13:
                                    Pdetec=0
                                PEarth = 1 - np.exp(-L_m1*5.06e13*(1/lambAbsm+1/Ldec+1/lambCompm)) #Extinction probability in this step
                                Ndec_res[t][j][r] += PEarth*(NDPs_tot[t][r]) #extinction number in this step
                                Ndec_tot[t][j][r] += Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j][r], 4)) #detection number in this step via either direct
                                                                                                        # interacion or decay  
                                
                                if j>0: 
                                    if NDPs_tot[t][r] < PEarth*(round(NDPs_tot[t][r], 4)):
                                        print(PEarth*(round(NDPs_tot[t][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 dec", j)
                                        Ndec_res[t][j][r] -= np.abs(NDPs_tot[t][r] - PEarth*(round(NDPs_tot[t][r], 4)))
                                        print(PEarth*(round(NDPs_tot[t][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 dec", j)
                                        Pdb.set_trace()
                                        
                                    if NDPs_tot[t][r] < Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j][r], 4)):
                                        print(Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 det", j)
                                        Ndec_tot[t][j][r] -= np.abs(NDPs_tot[t][r] - Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j][r], 4)))
                                        print(Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 det", j)
                                        Pdb.set_trace()
                                else:
                                    if NDPs_tot[t][r] < Ndec_res[t][j][r]:
                                        print(Ndec_res[t][j][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 dec")
                                        Ndec_res[t][j][r] -= (np.abs(Ndec_res[t][j][r] - NDPs_tot[t][r]))
                                        print(Ndec_res[t][j][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 dec")
                                        Pdb.set_trace()
                                    if NDPs_tot[t][r] < Ndec_tot[t][j][r]:
                                        print(Ndec_res[t][j][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 det")
                                        Ndec_tot[t][j][r] -= (np.abs(Ndec_tot[t][j][r] - NDPs_tot[t][r]))
                                        print(Ndec_res[t][j][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 det")
                                        Pdb.set_trace()

                                NDPs_tot[t][r] -= (Ndec_tot[t][j][r]+Ndec_res[t][j][r])
                                
                            else:
                                gamma = kappas**2*alpha1**4/(2**7*3**6*5**2*np.pi**3)*(mDP/1000)**9/me**8*(17/5+67/42*(mDP/1000)**2/me**2+128941/246960*(mDP/1000)**4/me**4)
                                Ldec = np.sqrt(E_z[r]**2/(mDP*1e-3)**2 - 1)/gamma
                                dist = (R_earth*np.sqrt(2*(1-np.cos(theta[t])))-x_m1[j])*100*5.06e13 #distance from step j to th detector ((GeV^{-1}))
                                Pdetec = np.exp(-dist*(1/Ldec+1/lambAbsm+1/lambCompm))*(1-np.exp(-40*5.06e13*(1/Ldec+1/lambAbsNaI+1/lambCompNaI))) #detection probability (0.4 m detector) 
                                if Pdetec<1e-13:
                                    Pdetec=0
                                PEarth = 1 - np.exp(-L_m1*5.06e13*(1/lambAbsm+1/Ldec+1/lambCompm)) #Extinction probability in this step
                                P=kappas**2*(mDP*1e-03)**4*funcmantle(Egamma*1000)*L_m1*5.06e13/(((mDP*1e-03)**2-mgamma_m**2)**2+Egamma**2*funcmantle(Egamma*1000)**2)
                                NOsc = P*(round(NDPs_tot[t][r], 4))#Oscillation
                                Ndec_res[t][j][r] += PEarth*(round(NDPs_tot[t][r]-NOsc, 4)) + NOsc #extinction number in this step
                                Ndec_tot[t][j][r] += Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j][r], 4))        #detection number in this step via either direct
                                                                                        # interacion or decay 
                                if j>0: 
                                    if NDPs_tot[t][r] < PEarth*(round(NDPs_tot[t][r], 4)):
                                        print(PEarth*(round(NDPs_tot[t][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 dec", j)
                                        Ndec_res[t][j][r] -= np.abs(NDPs_tot[t][r] - PEarth*(round(NDPs_tot[t][r], 4)))
                                        print(PEarth*(round(NDPs_tot[t][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 dec", j)
                                        Pdb.set_trace()
                                        
                                    if NDPs_tot[t][r] < Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j][r], 4)):
                                        print(Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 det", j)
                                        Ndec_tot[t][j][r] -= np.abs(NDPs_tot[t][r] - Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j][r], 4)))
                                        print(Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 det", j)
                                        Pdb.set_trace()
                                else:
                                    if NDPs_tot[t][r] < Ndec_res[t][j][r]:
                                        print(Ndec_res[t][j][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 dec")
                                        Ndec_res[t][j][r] -= (np.abs(Ndec_res[t][j][r] - NDPs_tot[t][r]))
                                        print(Ndec_res[t][j][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 dec")
                                        Pdb.set_trace()
                                    if NDPs_tot[t][r] < Ndec_tot[t][j][r]:
                                        print(Ndec_res[t][j][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 det")
                                        Ndec_tot[t][j][r] -= (np.abs(Ndec_tot[t][j][r] - NDPs_tot[t][r]))
                                        print(Ndec_res[t][j][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 det")
                                        Pdb.set_trace()
                                NDPs_tot[t][r] -= (Ndec_tot[t][j][r]+Ndec_res[t][j][r])
                                
            print("\n core, %f extinctions and %f detections" %(np.sum(Ndec_res),np.sum(Ndec_tot)))
            #In the core
            for j in range(0,len(x_c)):
                if (L_m+L_c) - x_c[j] >= 0:
                    for r in range(0, N):
                        if r!= 0:
                        
                            Egamma=(E_z[r]+E_z[r-1])/2 #GeV
                            lambAbsc = 1/(ne_c*sigAbsc[r])         #absorption length in core
                            lambAbsNaI = 1/(ne_NaI*sigAbsNaI[r])   #absorption length in NaI
                            lambCompc = 1/(ne_c*sigCompc[r])       #Compton length in core
                            lambCompNaI = 1/(ne_NaI*sigCompNaI[r]) #Compton length in NaI
                                
                            if mDP/1000 > 2*me: #Computing the number of decays + events near the detector
                                gamma=kappas**2*alpha1*(mDP*1e-3)/3*(1+2*me**2/(mDP*1e-3)**2)*(1-4*me**2/(mDP*1e-3)**2)**0.5
                                Ldec = np.sqrt(E_z[r]**2/(mDP*1e-3)**2 - 1)/gamma
                                dist = (R_earth*np.sqrt(2*(1-np.cos(theta[t])))-x_c[j])*100*5.06e13 #distance from step j to th detector ((GeV^{-1}))
                                Pdetec = np.exp(-dist*(1/Ldec+1/lambAbsc+1/lambCompc))*(1-np.exp(-40*5.06e13*(1/Ldec+1/lambAbsNaI+1/lambCompNaI))) #detection probability (0.4 m detector) 
                                if Pdetec<1e-13:
                                    Pdetec=0
                                PEarth = 1 - np.exp(-L_core*5.06e13*(1/lambAbsc+1/Ldec+1/lambCompc)) #Extinction probability in this step
                                Ndec_res[t][j+len(x_m1)][r] += PEarth*(round(NDPs_tot[t][r], 4)) #extinction number in this step
                                Ndec_tot[t][j+len(x_m1)][r] += Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j+len(x_m1)][r], 4)) #detection number in this step via either direct
                                                                                                                   # interacion or decay 
                                if j>0: 
                                    if NDPs_tot[t][r] < PEarth*(round(NDPs_tot[t][r], 4)):
                                        print(PEarth*(round(NDPs_tot[t][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 dec", j)
                                        Ndec_res[t][j+len(x_m1)][r] -= np.abs(NDPs_tot[t][r] - PEarth*(round(NDPs_tot[t][r], 4)))
                                        print(PEarth*(round(NDPs_tot[t][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 dec", j)
                                        Pdb.set_trace()
                                        
                                    if NDPs_tot[t][r] < Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j+len(x_m1)][r], 4)):
                                        print(Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j+len(x_m1)][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 det", j)
                                        Ndec_tot[t][j+len(x_m1)][r] -= np.abs(NDPs_tot[t][r] - Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j+len(x_m1)][r], 4)))
                                        print(Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j+len(x_m1)][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 det", j)
                                        Pdb.set_trace()
                                else:
                                    if NDPs_tot[t][r] < Ndec_res[t][j+len(x_m1)][r]:
                                        print(Ndec_res[t][j+len(x_m1)][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 dec")
                                        Ndec_res[t][j+len(x_m1)][r] -= (np.abs(Ndec_res[t][j+len(x_m1)][r] - NDPs_tot[t][r]))
                                        print(Ndec_res[t][j+len(x_m1)][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 dec")
                                        Pdb.set_trace()
                                    if NDPs_tot[t][r] < Ndec_tot[t][j+len(x_m1)][r]:
                                        print(Ndec_res[t][j+len(x_m1)][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 det")
                                        Ndec_tot[t][j+len(x_m1)][r] -= (np.abs(Ndec_tot[t][j][r] - NDPs_tot[t][r]))
                                        print(Ndec_res[t][j+len(x_m1)][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 det")
                                        Pdb.set_trace()

                                NDPs_tot[t][r] -= (Ndec_tot[t][j+len(x_m1)][r]+Ndec_res[t][j+len(x_m1)][r])
                                
                            else:
                                gamma = kappas**2*alpha1**4/(2**7*3**6*5**2*np.pi**3)*(mDP/1000)**9/me**8*(17/5+67/42*(mDP/1000)**2/me**2+128941/246960*(mDP/1000)**4/me**4)
                                Ldec = np.sqrt(E_z[r]**2/(mDP*1e-3)**2 - 1)/gamma
                                dist = (R_earth*np.sqrt(2*(1-np.cos(theta[t])))-x_c[j])*100*5.06e13 #distance from step j to th detector ((GeV^{-1}))
                                Pdetec = np.exp(-dist*(1/Ldec+1/lambAbsc+1/lambCompc))*(1-np.exp(-40*5.06e13*(1/Ldec+1/lambAbsNaI+1/lambCompNaI))) #detection probability (0.4 m detector) 
                                if Pdetec<1e-13:
                                    Pdetec=0
                                PEarth = 1 - np.exp(-L_core*5.06e13*(1/lambAbsc+1/Ldec+1/lambCompc)) #Extinction probability in this step
                                P=kappas**2*(mDP*1e-03)**4*funcore(Egamma*1000)*L_core*5.06e13/(((mDP*1e-03)**2-mgamma_c**2)**2+Egamma**2*funcore(Egamma*1000)**2)
                                NOsc = P*(round(NDPs_tot[t][r], 4)) #Oscillation
                                Ndec_res[t][j+len(x_m1)][r] += PEarth*(round(NDPs_tot[t][r]-NOsc, 4)) + NOsc #extinction number in this step
                                Ndec_tot[t][j+len(x_m1)][r] += Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j+len(x_m1)][r], 4)) #detection number in this step via either direct
                                                                                                                   # interacion or decay 
                                if j>0: 
                                    if NDPs_tot[t][r] < PEarth*(round(NDPs_tot[t][r], 4)):
                                        print(PEarth*(round(NDPs_tot[t][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 dec", j)
                                        Ndec_res[t][j+len(x_m1)][r] -= np.abs(NDPs_tot[t][r] - PEarth*(round(NDPs_tot[t][r], 4)))
                                        print(PEarth*(round(NDPs_tot[t][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 dec", j)
                                        Pdb.set_trace()
                                        
                                    if NDPs_tot[t][r] < Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j][r], 4)):
                                        print(Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j+len(x_m1)][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 det", j)
                                        Ndec_tot[t][j][r] -= np.abs(NDPs_tot[t][r] - Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j+len(x_m1)][r], 4)))
                                        print(Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j+len(x_m1)][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 det", j)
                                        Pdb.set_trace()
                                else:
                                    if NDPs_tot[t][r] < Ndec_res[t][j+len(x_m1)][r]:
                                        print(Ndec_res[t][j][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 dec")
                                        Ndec_res[t][j+len(x_m1)][r] -= (np.abs(Ndec_res[t][j+len(x_m1)][r] - NDPs_tot[t][r]))
                                        print(Ndec_res[t][j][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 dec")
                                        Pdb.set_trace()
                                    if NDPs_tot[t][r] < Ndec_tot[t][j+len(x_m1)][r]:
                                        print(Ndec_res[t][j+len(x_m1)][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 det")
                                        Ndec_tot[t][j+len(x_m1)][r] -= (np.abs(Ndec_tot[t][j+len(x_m1)][r] - NDPs_tot[t][r]))
                                        print(Ndec_res[t][j+len(x_m1)][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 det")
                                        Pdb.set_trace()

                                NDPs_tot[t][r] -= (Ndec_tot[t][j+len(x_m1)][r]+Ndec_res[t][j+len(x_m1)][r])

            print("\n mantle 2, %f extinctions and %f detections" %(np.sum(Ndec_res),np.sum(Ndec_tot)))
            #After coming out of the nucleus
            for j in range(0, len(x_m2)):
                if R_earth*np.sqrt(2*(1-np.cos(theta[t])))-x_m2[j]>=0:
                    for r in range(0, N):
                        if r!= 0:

                            Egamma=(E_z[r]+E_z[r-1])/2 #GeV
                            lambAbsm = 1/(ne_m*sigAbsm[r])         #absorption length in mantle
                            lambAbsNaI = 1/(ne_NaI*sigAbsNaI[r])   #absorption length in NaI
                            lambCompm = 1/(ne_m*sigCompm[r])       #Compton length in mantle
                            lambCompNaI = 1/(ne_NaI*sigCompNaI[r]) #Compton length in NaI

                            if mDP/1000 > 2*me: #Computing the number of decays + events near the detector
                                gamma=kappas**2*alpha1*(mDP*1e-3)/3*(1+2*me**2/(mDP*1e-3)**2)*(1-4*me**2/(mDP*1e-3)**2)**0.5
                                Ldec = np.sqrt(E_z[r]**2/(mDP*1e-3)**2 - 1)/gamma
                                dist = (R_earth*np.sqrt(2*(1-np.cos(theta[t])))-x_m2[j])*100*5.06e13 #distance from step j to th detector ((GeV^{-1}))
                                Pdetec = np.exp(-dist*(1/Ldec+1/lambAbsm+1/lambCompm))*(1-np.exp(-40*5.06e13*(1/Ldec+1/lambAbsNaI+1/lambCompNaI))) #detection probability (0.4 m detector) 
                                if Pdetec<1e-13:
                                    Pdetec=0
                                PEarth = 1 - np.exp(-L_m2*5.06e13*(1/lambAbsm+1/Ldec+1/lambCompm)) #Extinction probability in this step
                                Ndec_res[t][j+len(x_m1)+len(x_c)][r] += PEarth*(round(NDPs_tot[t][r], 4)) #extinction number in this step
                                Ndec_tot[t][j+len(x_m1)+len(x_c)][r] += Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j+len(x_m1)+len(x_c)][r], 4)) #detection number in this step via either direct
                                                                                                                            # interacion or decay  
                                if j>0: 
                                    if NDPs_tot[t][r] < PEarth*(round(NDPs_tot[t][r], 4)):
                                        print(PEarth*(round(NDPs_tot[t][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 dec", j)
                                        Ndec_res[t][j+len(x_m1)+len(x_c)][r] -= np.abs(NDPs_tot[t][r] - PEarth*(round(NDPs_tot[t][r], 4)))
                                        print(PEarth*(round(NDPs_tot[t][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 dec", j)
                                        Pdb.set_trace()
                                        
                                    if NDPs_tot[t][r] < Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j][r], 4)):
                                        print(Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j+len(x_m1)+len(x_c)][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 det", j)
                                        Ndec_tot[t][j+len(x_m1)+len(x_c)][r] -= np.abs(NDPs_tot[t][r] - Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j+len(x_m1)+len(x_c)][r], 4)))
                                        print(Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j+len(x_m1)+len(x_c)][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 det", j)
                                        Pdb.set_trace()
                                else:
                                    if NDPs_tot[t][r] < Ndec_res[t][j+len(x_m1)+len(x_c)][r]:
                                        print(Ndec_res[t][j+len(x_m1)+len(x_c)][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 dec")
                                        Ndec_res[t][j+len(x_m1)+len(x_c)][r] -= (np.abs(Ndec_res[t][j+len(x_m1)+len(x_c)][r] - NDPs_tot[t][r]))
                                        print(Ndec_res[t][j+len(x_m1)+len(x_c)][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 dec")
                                        Pdb.set_trace()
                                    if NDPs_tot[t][r] < Ndec_tot[t][j+len(x_m1)+len(x_c)][r]:
                                        print(Ndec_res[t][j+len(x_m1)+len(x_c)][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 det")
                                        Ndec_tot[t][j+len(x_m1)+len(x_c)][r] -= (np.abs(Ndec_tot[t][j+len(x_m1)+len(x_c)][r] - NDPs_tot[t][r]))
                                        print(Ndec_res[t][j+len(x_m1)+len(x_c)][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 det")
                                        Pdb.set_trace()

                                NDPs_tot[t][r] -= (Ndec_res[t][j+len(x_m1)+len(x_c)][r]+Ndec_tot[t][j+len(x_m1)+len(x_c)][r])
                                
                            else:
                                gamma = kappas**2*alpha1**4/(2**7*3**6*5**2*np.pi**3)*(mDP/1000)**9/me**8*(17/5+67/42*(mDP/1000)**2/me**2+128941/246960*(mDP/1000)**4/me**4)
                                Ldec = np.sqrt(E_z[r]**2/(mDP*1e-3)**2 - 1)/gamma
                                dist = (R_earth*np.sqrt(2*(1-np.cos(theta[t])))-x_m2[j])*100*5.06e13 #distance from step j to th detector ((GeV^{-1}))
                                Pdetec = np.exp(-dist*(1/Ldec+1/lambAbsm+1/lambCompm))*(1-np.exp(-40*5.06e13*(1/Ldec+1/lambAbsNaI+1/lambCompNaI))) #detection probability (0.4 m detector) 
                                if Pdetec<1e-13:
                                    Pdetec=0
                                PEarth = 1 - np.exp(-L_m2*5.06e13*(1/lambAbsm+1/Ldec+1/lambCompm)) #Extinction probability in this step
                                P=kappas**2*(mDP*1e-03)**4*funcmantle(Egamma*1000)*L_m2*5.06e13/(((mDP*1e-03)**2-mgamma_m**2)**2+Egamma**2*funcmantle(Egamma*1000)**2)
                                NOsc = P*(round(NDPs_tot[t][r], 4)) #Oscillation
                                Ndec_res[t][j+len(x_m1)+len(x_c)][r] += PEarth*(round(NDPs_tot[t][r]-NOsc, 4)) + NOsc #extinction number in this step
                                Ndec_tot[t][j+len(x_m1)+len(x_c)][r] += Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j+len(x_m1)+len(x_c)][r], 4)) #detection number in this step via either direct
                                                                                                                            # interacion or decay 
                                if j>0: 
                                    if NDPs_tot[t][r] < PEarth*(round(NDPs_tot[t][r], 4)):
                                        print(PEarth*(round(NDPs_tot[t][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 dec", j)
                                        Ndec_res[t][j+len(x_m1)+len(x_c)][r] -= np.abs(NDPs_tot[t][r] - PEarth*(round(NDPs_tot[t][r], 4)))
                                        print(PEarth*(round(NDPs_tot[t][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 dec", j)
                                        Pdb.set_trace()
                                        
                                    if NDPs_tot[t][r] < Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j][r], 4)):
                                        print(Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j+len(x_m1)+len(x_c)][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 det", j)
                                        Ndec_tot[t][j+len(x_m1)+len(x_c)][r] -= np.abs(NDPs_tot[t][r] - Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j+len(x_m1)+len(x_c)][r], 4)))
                                        print(Pdetec*(round(NDPs_tot[t][r]-Ndec_res[t][j+len(x_m1)+len(x_c)][r], 4)), NDPs_tot[t][r], Pdetec, PEarth, "j>0 det", j)
                                        Pdb.set_trace()
                                else:
                                    if NDPs_tot[t][r] < Ndec_res[t][j+len(x_m1)+len(x_c)][r]:
                                        print(Ndec_res[t][j][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 dec")
                                        Ndec_res[t][j+len(x_m1)+len(x_c)][r] -= (np.abs(Ndec_res[t][j+len(x_m1)+len(x_c)][r] - NDPs_tot[t][r]))
                                        print(Ndec_res[t][j][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 dec")
                                        Pdb.set_trace()
                                    if NDPs_tot[t][r] < Ndec_tot[t][j+len(x_m1)+len(x_c)][r]:
                                        print(Ndec_res[t][j+len(x_m1)+len(x_c)][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 det")
                                        Ndec_tot[t][j][r] -= (np.abs(Ndec_tot[t][j+len(x_m1)+len(x_c)][r] - NDPs_tot[t][r]))
                                        print(Ndec_res[t][j+len(x_m1)+len(x_c)][r], NDPs_tot[t][r], Pdetec, PEarth, "j=0 det")
                                        Pdb.set_trace()
                                        
                                NDPs_tot[t][r] -= (Ndec_tot[t][j+len(x_m1)+len(x_c)][r]+Ndec_res[t][j+len(x_m1)+len(x_c)][r])

    print("Remaining DPs: %f" %(np.sum(NDPs_tot)))               
    return(Ndec_res, Ndec_tot)

In [11]:
def CalcOsc(theta, NDPs_tot, kappas, mDP, Na, funcNaI):

    alpha1 = 1/137
    N = 50
    me=0.511/1000 #electron mass (GeV)
    rho_NaI = 3.67
    ne_NaI=rho_NaI/(149.89/Na)*(1.973*10**(-14))**3   #Electron number density for NaI crystals (in GeV^3)

    for t in range(0,len(theta)): #angles
        for r in range(0, N):
            if r!= 0:
                NDPs_tot[t][r] = round(NDPs_tot[t][r], 1)

                        # if NDPs_tot[t][j][r] < 0:
                        #     print(NDPs_tot[t][j][r])
                        #     NDPs_tot[t][j][r] = 0
    #NDPs_tot = round(np.sum(NDPs_tot), 4)

    #NOW WE COMPUTE THE INTERACTIONS WITH THE CRYSTAL
    detectheta = np.arcsin(np.sin(theta)/(np.sqrt(2*(1-np.cos(theta)))))
    xdetect = np.linspace(0,0.4/np.cos(detectheta[len(detectheta)-1]),500) #Steps inside the NaI crystals (m)
    NOsc_tot = np.zeros((len(detectheta), len(xdetect), N)) #Number of oscillations in each angle, step and energy
    Ldetec = np.diff(xdetect)[0] 
    mgamma_NaI=(4*np.pi*alpha1*ne_NaI/me)**0.5    #effective photon mass (GeV)

    if (mDP>1E-03):
        E_z = np.linspace(mDP, 3, N) #MeV
                    
    else:
        E_z = np.linspace(1E-03, 3, N) #MeV

    for t in range(0,len(theta)):
        print("detection", t)
        for j in range(0,len(xdetect)-1):
            if ((0.4/np.cos(theta[t])-xdetect[j])>=0.0):

                for r in range(N):

                    #Oscillation
                    Egamma=(E_z[r]+E_z[r-1])/2 #MeV
                    P=kappas**2*mDP**4*funcNaI(Egamma)*1e03*Ldetec*5.06e10/((mDP**2-(mgamma_NaI*1e03)**2)**2+Egamma**2*(funcNaI(Egamma)*1e03)**2)
                    NOsc = round(NDPs_tot[t][r],3)*P
                    NOsc_tot[t][j][r] += NOsc 
                    NDPs_tot[t][r] -= NOsc

                    if j>0:
                        if round(NDPs_tot[t][r],3) < np.sum(NOsc_tot[t][:j,r]):
                            print("j>0, Osc", P, NOsc, j)
                            # exit()
                            NOsc_tot[t][j][r] -= NOsc
                    else:
                        if round(NDPs_tot[t][r],3) < NOsc_tot[t][j][r]:
                            print("j>0, Osc", P, NOsc, j)
                            # exit()
                            NOsc_tot[t][j][r] -= NOsc
    return(NOsc_tot)

In [11]:
mDP = 0.000121 #MeV
kappas = 0.00133352143
# mDP = 1.7 #MeV
# kappas = 0.00006493816
epsilon = 1e-4
R_earth = 6371000   #Earth's radius in meters

# Import vertical muon flux from around 0.8 GeV to few TeV
arq = open('/home/davidc/Documents/Master\'s Analisys/Parameter space/muonflux.tsv', 'r')
#arq = open('/home/dfreitas/DPnum_Comp/Muons/muonflux.tsv', 'r')
texto1 = arq.readlines()
arq.close()

texto1 = ExtractData(texto1)

N1=len(texto1)
p=np.zeros(N1)    #muon momentum
y=np.zeros(N1)
#E=np.zeros(N1)   #muon energy
mu=0.105658 #GeV  #muon mass
Na=6.02*10**23    #Avogadro number

for i in range(N1):
    p[i]=texto1[i][0]
    y[i]=texto1[i][1]
    #E[i]=(p[i]**2+mu**2)**0.5 # Since p>>mu, E~p
   
# Interpolate muon flux in units of muons/cm^2/s/sr
func=inter.interp1d(p,y/p**3)

# Import muon stopping power in rock
arq = open('/home/davidc/Documents/Master\'s Analisys/Parameter space/muon_stoppower.tsv', 'r')
#arq = open('/home/dfreitas/DPnum_Comp/Muons/muon_stoppower.tsv', 'r')
texto1 = arq.readlines()
arq.close()

texto1 = ExtractData(texto1)

N1=len(texto1)
dEdx=np.zeros(N1) #Energy variation rate for the muon in each step
Em=np.zeros(N1)   

Build_general(texto1, Em, dEdx, N1)

funcdEdx=inter.interp1d(Em,dEdx)

# Import muon flux variation with the zenith angle for 1 GeV muons
arq = open('/home/davidc/Documents/Master\'s Analisys/Parameter space/muonangle_1GeV.tsv', 'r')
#arq = open('/home/dfreitas/DPnum_Comp/Muons/muonangle_1GeV.tsv', 'r')
texto1 = arq.readlines()
arq.close()

texto1 = ExtractData(texto1)

N1=len(texto1)
zenite_1GeV=np.zeros(N1)
ratio_1GeV=np.zeros(N1)

Build_zenital(texto1, zenite_1GeV, ratio_1GeV, N1)

func1GeV=inter.interp1d(zenite_1GeV,ratio_1GeV)

# Import muon flux variation with the zenith angle for 10 GeV muons
arq = open('/home/davidc/Documents/Master\'s Analisys/Parameter space/muonangle_10GeV.tsv', 'r')
#arq = open('/home/dfreitas/DPnum_Comp/Muons/muonangle_10GeV.tsv', 'r')
texto1 = arq.readlines()
arq.close()

texto1 = ExtractData(texto1)

N1=len(texto1)
zenite_10GeV=np.zeros(N1)
ratio_10GeV=np.zeros(N1)

Build_zenital(texto1, zenite_10GeV, ratio_10GeV, N1)

func10GeV=inter.interp1d(zenite_10GeV,ratio_10GeV)

# Import muon flux variation with the zenith angle for 100 GeV muons
arq = open('/home/davidc/Documents/Master\'s Analisys/Parameter space/muonangle_100GeV.tsv', 'r')
#arq = open('/home/dfreitas/DPnum_Comp/Muons/muonangle_100GeV.tsv', 'r')
texto1 = arq.readlines()
arq.close()

texto1 = ExtractData(texto1)

N1=len(texto1)
zenite_100GeV=np.zeros(N1)
ratio_100GeV=np.zeros(N1)

Build_zenital(texto1, zenite_100GeV, ratio_100GeV, N1)

func100GeV=inter.interp1d(zenite_100GeV,ratio_100GeV)

# Import muon flux variation with the zenith angle for 1 TeV muons
arq = open('/home/davidc/Documents/Master\'s Analisys/Parameter space/muonangle_1000GeV.tsv', 'r')
#arq = open('/home/dfreitas/DPnum_Comp/Muons/muonangle_1000GeV.tsv', 'r')
texto1 = arq.readlines()
arq.close()

texto1 = ExtractData(texto1)

N1=len(texto1)
zenite_1000GeV=np.zeros(N1)
ratio_1000GeV=np.zeros(N1)

Build_zenital(texto1, zenite_1000GeV, ratio_1000GeV, N1)

func1000GeV=inter.interp1d(zenite_1000GeV,ratio_1000GeV)

# Interpolate muon flux variation with zenith for different muon energies
theta_earth = np.linspace(3.0*np.pi/180,177*np.pi/180,100) # Earth's theta angle in spherical coordinates, considering n degrees resolution
    
#print(theta*180/np.pi)
en=np.logspace(0,3,4)
interen=np.zeros(4)
Ei=np.logspace(3,6.69897,1000) #Muon energies from 1 GeV to few TeV (in MeV)
#print((Ei[0]-np.diff(Ei)[0]/2)/1000)
ratio=np.zeros((len(theta_earth),len(Ei)))

for t in range(len(theta_earth)):
    dvec = np.array([-R_earth*np.sin(theta_earth[t]),0,R_earth*(1-np.cos(theta_earth[t]))]) #string vector in the sphere (phi=0)
    rhat = np.array([np.sin(theta_earth[t]),0,np.cos(theta_earth[t])])                      #position unit vector in the sphere (phi=-0)
    zenite = np.arccos(np.dot(-dvec,rhat)/(R_earth*np.sqrt(2*(1-np.cos(theta_earth[t])))))  #zenith angle at some position
    interen[0]=func1GeV(zenite)
    interen[1]=func10GeV(zenite)
    interen[2]=func100GeV(zenite)
    interen[3]=func1000GeV(zenite)
    funcen=inter.interp1d(en,interen)
    for i in range(len(Ei)):
        if (Ei[i]<=1E+6):
            ratio[t][i]=funcen(Ei[i]/1000)
        else:
            ratio[t][i]=2.0*funcen(1E+6/1000) # Since the muon flux variation with zenith is only given to energies up to 1 TeV, consider that for energies above
                                                # 1 TeV the muon flux is two times the muon flux for 1 TeV.

#x=np.linspace(0,(700/np.cos(theta[14]))/700*1855*100,5000) # Define step length in cm used in the muon attenuation and DP generated by Bremsstrahlung in the rock.
x=np.linspace(0,14000*2.65*100,6000) # Define step length in cm used in the muon attenuation and DP generated by Bremsstrahlung in the rock.

# Calculate muon attenuation when travelling in the rock
E=np.zeros((len(Ei),len(x)))
for i in range(len(Ei)):
        
    for j in range(len(x)-1):
        if x[j]/2.65/100 <= 14000: #Rock density = 2.65 g/cm³ and muons penetrate till' 14000 m max
            if (j==0):
                E[i][j]=Ei[i]

            if (E[i][j]<=1.2): continue
            E[i][j+1]=E[i][j]-funcdEdx(E[i][j])*(x[j+1]-x[j]) 

Acosine=(2*100)**2 #cm^2 COSINE-100 LS Area
Nmu=np.zeros((len(theta_earth),len(Ei)-1))
d = R_earth*np.sqrt(2*(1-np.cos(theta_earth)))
dOmega = R_earth**2/d*np.sin(theta_earth)*np.cos(np.arcsin((R_earth/d*np.sin(theta_earth))))*np.cos(theta_earth)/np.sqrt(d**2 - (R_earth*np.sin(theta_earth))**2)
dOmegafunc = inter.interp1d(theta_earth, dOmega, fill_value="extrapolate")

# Check how many muons had a trajectory that would cross COSINE LS detector
for i in range(len(Ei)-1):

    for t in range(len(theta_earth)-1): 
        I = quad(func, (Ei[i]-np.diff(Ei)[i]/2)/1000, (Ei[i]+np.diff(Ei)[i]/2)/1000)
        sr = 2*np.pi*quad(dOmegafunc, (theta_earth[t]-np.diff(theta_earth)[t])/1000, (theta_earth[t]+np.diff(theta_earth)[t])/1000)[0]
        #sr = Acosine/(2*(R_earth*100)**2*(1-np.cos(theta_earth[t]))) #Omega = Area/distance²
        Nmu[t][i] = ratio[t][i]*I[0]*Acosine*sr #Muons/s

# Check how many muons there are in each zenith angle, depth in rock, and with energy E[i][j]. 
# It is required that the muon energy has to be greater than 100 MeV.
    
Nmuabove100=np.zeros((len(Ei)-1,len(theta_earth),len(x)))
 
for i in range(len(Ei)-1):
    for t in range(len(theta_earth)):
        j=0
        while (E[i][j]>1.2 and x[j]/2.65/100<14000):
            if (E[i][j]>1*10**2):
                Nmuabove100[i][t][j]=Nmu[t][i]*2*np.pi
            j=j+1

print(np.sum(Nmuabove100))
#Building the mass attenuation function for each rock material
(funcSi, en_Si)=InterpMassAtt("Si")
(funcAl, en_Al)=InterpMassAtt("Al")
(funcO, en_O)=InterpMassAtt("O")
(funcFe, en_Fe)=InterpMassAtt("Fe")
(funcCa, en_Ca)=InterpMassAtt("Ca")
(funcMg, en_Mg)=InterpMassAtt("Mg")
(funcNi, en_Ni)=InterpMassAtt("Ni")
(funcI, en_I)=InterpMassAtt("I")
(funcNa, en_Na)=InterpMassAtt("Na")
    
en1_mantle = GatherEnergies(en_Al, en_Si)
en2_mantle = GatherEnergies(en1_mantle, en_O)  
en3_mantle = GatherEnergies(en2_mantle, en_Fe)
en4_mantle = GatherEnergies(en3_mantle, en_Ca)
enfinal_mantle = GatherEnergies(en4_mantle, en_Mg)

enfinal_core = GatherEnergies(en_Fe, en_Ni)
enfinal_NaI = GatherEnergies(en_Na, en_I)

att_mantle=np.zeros(len(enfinal_mantle))  
for i in range(len(enfinal_mantle)):#doi:10.1016/j.epsl.2004.12.005
    att_mantle[i]= 0.2*funcSi(enfinal_mantle[i])+0.448*funcO(enfinal_mantle[i])+0.03*funcAl(enfinal_mantle[i])
    +0.08*funcFe(enfinal_mantle[i])+0.014*funcCa(enfinal_mantle[i])+0.228*funcMg(enfinal_mantle[i])
funcmantle=inter.interp1d(enfinal_mantle, att_mantle*3.67*(1.973*10**(-14)), kind='linear', fill_value="extrapolate") #GeV, with energy given in MeV # Mass attenuatioin for the rock.

att_core=np.zeros(len(enfinal_core))  
for i in range(len(enfinal_core)):#IAG (Eder Molina:www.iag.usp.br/~eder/estrutura_e_evolucao.pdf)
    att_core[i]= 0.9*funcFe(enfinal_core[i])+0.1*funcNi(enfinal_core[i])
funccore=inter.interp1d(enfinal_core, att_core*3.67*(1.973*10**(-14)), kind='linear', fill_value="extrapolate") #GeV, with energy given in MeV # Mass attenuatioin for the rock.

att_NaI=np.zeros(len(enfinal_NaI))  
for i in range(len(enfinal_NaI)):
    att_NaI[i]= 0.5*funcNa(enfinal_NaI[i])+0.5*funcI(enfinal_NaI[i])
funcNaI=inter.interp1d(enfinal_NaI, att_NaI*3.67*(1.973*10**(-14)), kind='linear', fill_value="extrapolate") #GeV, with energy given in MeV # Mass attenuatioin for the rock.

6453.974826306122


In [12]:
#Computing the total number of Bremsstrahlungs
NDPs_tot = MuonCalc(Ei, theta_earth, E, mDP, x, kappas, Nmuabove100, Na)
N = len(NDPs_tot[0])
NDPsAll = np.zeros(N)
for i in range(0, N):
    NDPsAll[i] = round(np.sum(NDPs_tot[:,i]),4)

with open('/home/davidc/Documents/Master\'s Analisys/Parameter space/Codes/Direct Interaction/NDet_int_m=%.8f_k=%.11f.txt' %(mDP,kappas), 'w') as archive:
    for i in range(0, N):
        archive.write('{:.16f}\n'.format(NDPsAll[i]))


Bremsstrahlung = 0.000000
 0
Bremsstrahlung = 0.351893
 1
Bremsstrahlung = 0.711437
 2
Bremsstrahlung = 1.078789
 3
Bremsstrahlung = 1.453584
 4
Bremsstrahlung = 1.835090
 5
Bremsstrahlung = 2.223371
 6
Bremsstrahlung = 2.618389
 7
Bremsstrahlung = 3.018712
 8
Bremsstrahlung = 3.423853
 9
Bremsstrahlung = 3.833778
 10
Bremsstrahlung = 4.248482
 11
Bremsstrahlung = 4.670541
 12
Bremsstrahlung = 5.101734
 13
Bremsstrahlung = 5.542161
 14
Bremsstrahlung = 5.990681
 15
Bremsstrahlung = 6.446886
 16
Bremsstrahlung = 6.910772
 17
Bremsstrahlung = 7.380910
 18
Bremsstrahlung = 7.856730
 19
Bremsstrahlung = 8.338202
 20
Bremsstrahlung = 8.825292
 21
Bremsstrahlung = 9.318029
 22
Bremsstrahlung = 9.816423
 23
Bremsstrahlung = 10.320434
 24
Bremsstrahlung = 10.830016
 25
Bremsstrahlung = 11.351227
 26
Bremsstrahlung = 11.881404
 27
Bremsstrahlung = 12.421516
 28
Bremsstrahlung = 12.971776
 29
Bremsstrahlung = 13.531582
 30
Bremsstrahlung = 14.099949
 31
Bremsstrahlung = 14.676989
 32
Bremsstrahl

In [16]:
#Computing the number of events in Earth and NaI
(Ndec_res, Ndec_tot) = CalcEvents(NDPs_tot, theta_earth, mDP, kappas, Na, epsilon, funcmantle, funccore, R_earth)

events 0

mantle only, 0.000000 extinctions and 0.000000 detections
events 1

mantle only, 0.000165 extinctions and 0.000000 detections
events 2

mantle only, 0.000427 extinctions and 0.000000 detections
events 3

mantle only, 0.000774 extinctions and 0.000000 detections
events 4

mantle only, 0.001196 extinctions and 0.000000 detections
events 5

mantle only, 0.001711 extinctions and 0.000000 detections
events 6

mantle only, 0.002324 extinctions and 0.000000 detections
events 7

mantle only, 0.003034 extinctions and 0.000000 detections
events 8

mantle only, 0.003846 extinctions and 0.000000 detections
events 9

mantle only, 0.004769 extinctions and 0.000000 detections
events 10

mantle only, 0.005810 extinctions and 0.000000 detections
events 11

mantle only, 0.006969 extinctions and 0.000000 detections
events 12

mantle only, 0.008235 extinctions and 0.000000 detections
events 13

mantle only, 0.009610 extinctions and 0.000000 detections
events 14

mantle only, 0.011111 extinctions

In [17]:
#Number of oscillations inside the crystals
if mDP < 2*0.511:
    NOsc_tot = CalcOsc(theta_earth, NDPs_tot, kappas, mDP, Na, funcNaI)

In [18]:
N = len(NDPs_tot[0])
# NCompsNaI = np.zeros(N)
# NAbsNaI = np.zeros(N)
# NOscNaI = np.eros(N)
NdecNaI = np.zeros(N)
NDPsNaI = np.zeros(N)
Ndec = np.zeros(N)

for i in range(0, N):
    # NCompsNaI[i] = round(np.sum(NComps_tot[:,:,i]),4)
    # NAbsNaI[i] = round(np.sum(NAbs_tot[:,:,i]),4)
    # NOscNaI[i] = round(np.sum(NOsc_tot[:,:,i]),4)
    NdecNaI[i] = round(np.sum(Ndec_tot[:,:,i]),4)
    NDPsNaI[i] = round(np.sum(NDPs_tot[:,i]),4)
    Ndec[i] = round(np.sum(Ndec_res[:,:,i]),4)

if (mDP>1E-03):
    E_z = np.linspace(mDP, 3, N) #MeV
                    
else:
    E_z = np.linspace(1E-03, 3, N) #MeV
      
# with open('/home/davidc/Documents/Master\'s Analisys/Parameter space/Codes/Direct Interaction/NDet_int_m=%.8f_k=%.11f.txt' %(mDP,kappas), 'w') as archive:
#     for i in range(0, N):
#         archive.write('{:.11f}  {:.16f}  {:.16f}  {:.16f}  {:.16f}\n'.format(E_z[i], NCompsNaI[i], NAbsNaI[i], NOscNaI[i], NdecNaI[i]))

# with open('/home/dfreitas/DPnum_Comp/DirDetec/Spectra/NDir_int_m=%.8f_k=%.11f.txt' %(mDP,kappas), 'w') as archive:
#     for i in range(0, N):
#         archive.write('{:.11f}  {:.16f}  {:.16f}  {:.16f}  {:.16f}\n'.format(E_z[i], NCompsNaI[i], NAbsNaI[i], NOscNaI[i], NdecNaI[i]))

print("total DPs = \n", np.sum(NDPs_tot))
#print("\nNumber of Bremsstrahlung producing DPs: \n", totalDPs)
print("\nNumber of detections:\n", np.sum(Ndec_tot))
print("\nNumber of events inside Earth:\n", np.sum(Ndec_res))
print("\nNumber of oscillations inside crystals: \n", np.sum(NOsc_tot))
#print("\nNumber of Comptons processes: \n", "{:e}".format(np.sum(NComps_res)))
#print("\nPorcentagem de DP's que sofrem Compton: %d" %(np.sum(NComps_res)/np.sum(NDPs_tot)))


total DPs = 
 0.0

Number of detections:
 0.0

Number of events inside Earth:
 0.8243050848730781


NameError: name 'NOsc_tot' is not defined